In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import scml
from scml import pandasx as pdx
from mylib.ner import NerModel
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:1")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

int16, min=-32768, max=32767
device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [2]:
dst_path = Path("models/ner/deberta_v3_base/20240409_053545")
ckpt_path = dst_path / "lightning_logs/version_0/checkpoints/epoch=0-step=1001-val_loss=0.02627.ckpt"
checkpoint = torch.load(ckpt_path)
print(checkpoint.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])


In [3]:
print(checkpoint["state_dict"].keys())

odict_keys(['model.deberta.embeddings.word_embeddings.weight', 'model.deberta.embeddings.LayerNorm.weight', 'model.deberta.embeddings.LayerNorm.bias', 'model.deberta.encoder.layer.0.attention.self.query_proj.weight', 'model.deberta.encoder.layer.0.attention.self.query_proj.bias', 'model.deberta.encoder.layer.0.attention.self.key_proj.weight', 'model.deberta.encoder.layer.0.attention.self.key_proj.bias', 'model.deberta.encoder.layer.0.attention.self.value_proj.weight', 'model.deberta.encoder.layer.0.attention.self.value_proj.bias', 'model.deberta.encoder.layer.0.attention.output.dense.weight', 'model.deberta.encoder.layer.0.attention.output.dense.bias', 'model.deberta.encoder.layer.0.attention.output.LayerNorm.weight', 'model.deberta.encoder.layer.0.attention.output.LayerNorm.bias', 'model.deberta.encoder.layer.0.intermediate.dense.weight', 'model.deberta.encoder.layer.0.intermediate.dense.bias', 'model.deberta.encoder.layer.0.output.dense.weight', 'model.deberta.encoder.layer.0.output.

In [4]:
%%time
model = NerModel.load_from_checkpoint(ckpt_path)

[INFO|ner.py:413] 2024-04-10 15:48:08,064 >> config.to_diff_dict={
  "id2label": {
    "0": "O",
    "1": "B-NAME_STUDENT",
    "2": "I-NAME_STUDENT",
    "3": "B-URL_PERSONAL",
    "4": "I-URL_PERSONAL",
    "5": "B-ID_NUM",
    "6": "I-ID_NUM",
    "7": "B-EMAIL",
    "8": "I-EMAIL",
    "9": "B-USERNAME",
    "10": "I-USERNAME",
    "11": "B-PHONE_NUM",
    "12": "I-PHONE_NUM",
    "13": "B-STREET_ADDRESS",
    "14": "I-STREET_ADDRESS"
  },
  "label2id": {
    "O": 0,
    "B-NAME_STUDENT": 1,
    "I-NAME_STUDENT": 2,
    "B-URL_PERSONAL": 3,
    "I-URL_PERSONAL": 4,
    "B-ID_NUM": 5,
    "I-ID_NUM": 6,
    "B-EMAIL": 7,
    "I-EMAIL": 8,
    "B-USERNAME": 9,
    "I-USERNAME": 10,
    "B-PHONE_NUM": 11,
    "I-PHONE_NUM": 12,
    "B-STREET_ADDRESS": 13,
    "I-STREET_ADDRESS": 14
  },
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "_name_or_path": "/mnt/c/huggingface/microsoft/deberta-v3-base",
  "transformers_version": "4.37.0",
  "model_type": "deberta-v2"

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at /mnt/c/huggingface/microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 11 s, sys: 1.3 s, total: 12.3 s
Wall time: 14.5 s


In [5]:
%%time
model.model.save_pretrained(str(dst_path))

CPU times: user 923 ms, sys: 164 ms, total: 1.09 s
Wall time: 3.26 s


In [6]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:29.252839
